In [16]:
import os
import numpy as np
import netCDF4
import datetime
import logging
import cmocean
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import sdcclimatologies

In [2]:
from importlib import reload
reload(sdcclimatologies)

<module 'sdcclimatologies' from '/home/ctroupin/Projects/SeaDataCloud/Python/sdcclimatologies.py'>

In [17]:
logger = logging.getLogger("WOA-Plots")
logger.setLevel(logging.INFO)
logging.info("Starting")

## Files and directories

In [20]:
baseurl = "https://data.nodc.noaa.gov/thredds/dodsC/ncei/woa/"
wod_annual_T = sdcclimatologies.get_WOA_url("temperature", 0)
wod_annual_S = sdcclimatologies.get_WOA_url("salinity", 0)

sdcdir = "../Julia/Climatologies/NorthSea/output/"
sdc_annual_T = os.path.join(sdcdir, "Water_body_Temperature_NorthSea.4Danl_annual_merged.nc")
sdc_annual_S = os.path.join(sdcdir, "Water_body_Salinity_NorthSea.4Danl_annual_merged.nc")

figdir = "/home/ctroupin/dox/DIVAnd-data/NorthSea/figures/figures/WOA/annual/"
figdir2 = "/home/ctroupin/dox/DIVAnd-data/NorthSea/figures/SDC/annual/"
if not os.path.exists(figdir):
    os.makedirs(figdir)
if not os.path.exists(figdir2):
    os.makedirs(figdir2)

## Load coordinates from WOA

In [21]:
lonWOA, latWOA, depthWOA, timeWOA, timeWOAunits = \
sdcclimatologies.get_WOA_coords(wod_annual_T)

## Select the fields for the region of interest
Load the coordinates from the SDN product and find the good WOA coordinates.

In [22]:
lonmin, lonmax, latmin, latmax, dmin, dmax, datemin, datemax = \
sdcclimatologies.get_SDN_domain(sdc_annual_T)

In [23]:
goodlon = (lonWOA >= lonmin - 1.) & (lonWOA <= lonmax + 1.)
goodlat = (latWOA >= latmin - 1.) & (latWOA <= latmax + 1.)
gooddepth = (depthWOA >= dmin) & (depthWOA <= dmax)
lonWOA = lonWOA[goodlon]
latWOA = latWOA[goodlat]
depthWOA = depthWOA[gooddepth]

### Load the WOA field in the area of interest

In [8]:
with netCDF4.Dataset(wod_annual_T) as nc:
    T_WOA = nc.variables["t_an"][0, gooddepth, goodlat, goodlon]
with netCDF4.Dataset(wod_annual_S) as nc:
    S_WOA = nc.variables["s_an"][0, gooddepth, goodlat, goodlon]

## Make the plots
### Define projection

In [24]:
m = Basemap(projection='merc',llcrnrlat=latmin, urcrnrlat=latmax,
            llcrnrlon=lonmin, urcrnrlon=lonmax, lat_ts=0.5*(latmin+latmax), resolution='i')

### Temperature WOA

In [10]:
varname = "temperature"
for idepth, depth in enumerate(depthWOA):
    
    fname = "WOA_{}_annual_{}m".format(varname, str(int(depth)).zfill(3))
    logger.debug("Plotting in figure: {}".format(fname))
    figname = os.path.join(figdir, fname)
    
    logger.debug("Working on depth: {} m".format(depth))
    sdcclimatologies.make_2Dplot(m, lonWOA, latWOA, T_WOA[idepth,:,:], varname=varname,
                depth=depth, product="WOA annual", figname=figname,
                vmin=7.5, vmax=15)
    plt.close()
    
logger.info("Figures saved in {}".format(figdir))

INFO:WOA-Plots:Figures saved in ../figures/WOA/annual/


### Temperature SDC

In [25]:
lonSDC, latSDC, depthSDC, datesSDC, T_SDC = \
sdcclimatologies.read_climato_sdc(sdc_annual_T, "temperature")
T_SDC = T_SDC.squeeze()

In [26]:
varname = "temperature"
for idepth, depth in enumerate(depthSDC):
    
    fname = "SDC_{}_annual_{}m".format(varname, str(int(depth)).zfill(3))
    logger.debug("Plotting in figure: {}".format(fname))
    figname = os.path.join(figdir2, fname)
    
    logger.debug("Working on depth: {} m".format(depth))
    sdcclimatologies.make_2Dplot(m, lonSDC, latSDC, T_SDC[idepth,:,:], varname=varname,
                depth=depth, product="SDC annual", figname=figname,
                vmin=7.5, vmax=15)
    plt.close()
    
logger.info("Figures saved in {}".format(figdir))

INFO:WOA-Plots:Figures saved in /home/ctroupin/dox/DIVAnd-data/NorthSea/figures/figures/WOA/annual/


### Salinity WOA

In [13]:
varname = "salinity"
for idepth, depth in enumerate(depthWOA):
    
    fname = "WOA_{}_annual_{}m".format(varname, str(int(depth)).zfill(3))
    logger.debug("Plotting in figure: {}".format(fname))
    figname = os.path.join(figdir, fname)
    
    logger.debug("Working on depth: {} m".format(depth))
    sdcclimatologies.make_2Dplot(m, lonWOA, latWOA, S_WOA[idepth,:,:], varname=varname,
                depth=depth, product="WOA annual", figname=figname,
                vmin=32., vmax=36.)
    plt.close()
    
logger.info("Figures saved in {}".format(figdir))

INFO:WOA-Plots:Figures saved in ../figures/WOA/annual/


### Salinity SDC

In [29]:
lonSDC, latSDC, depthSDC, datesSDC, S_SDC = \
sdcclimatologies.read_climato_sdc(sdc_annual_S, "salinity")

In [32]:
S_SDC = S_SDC.squeeze()

In [33]:
varname = "salinity"
for idepth, depth in enumerate(depthSDC):
    
    fname = "SDC_{}_annual_{}m".format(varname, str(int(depth)).zfill(3))
    logger.debug("Plotting in figure: {}".format(fname))
    figname = os.path.join(figdir2, fname)
    
    logger.debug("Working on depth: {} m".format(depth))
    sdcclimatologies.make_2Dplot(m, lonSDC, latSDC, S_SDC[idepth,:,:], varname=varname,
                depth=depth, product="SDC annual", figname=figname,
                vmin=32., vmax=36.)
    plt.close()
    
logger.info("Figures saved in {}".format(figdir))

INFO:WOA-Plots:Figures saved in /home/ctroupin/dox/DIVAnd-data/NorthSea/figures/figures/WOA/annual/
